# More Examples

This notebook contains more examples, including a [larger-scale example with 1 million points](#Large-Scale-Test).

In [1]:
import numpy as np
import pandas as pd
import jscatter

We'll start by creating some dummy data that we can use to test jupyter-scatter.

In [2]:
n = 500
data = np.random.rand(n, 5)
df = pd.DataFrame(data, columns=['x', 'y', 'value', 'group', 'bla'])
df['group'] = df['x'].map(lambda x: chr(65 + round(x * 7)))
df.head(5)

,x,y,value,group,bla
0,0.519439,0.533727,0.289151,E,0.841719
1,0.701327,0.544183,0.905323,F,0.465165
2,0.147098,0.972666,0.554893,B,0.868960
3,0.280768,0.414519,0.131672,C,0.628633
4,0.282146,0.162423,0.046793,C,0.112329


## Simple Test

In this example, we'll demonstrate the functional API.

In [3]:
scatter = jscatter.Scatter('x', 'y', df)
scatter.show()

A useful feature of ipywidgets is that you can change the scatter plot after displaying it. Using the functional API we can chain multiple changes together.

In [4]:
scatter.color(by='group').size(6).height(320).selection(df.query('x < 0.5').index).axes(grid=True, labels=['X', 'Y'])

Jupyter-scatter provides sensible default configurations. For instance, in the example above jupyter-scatter employs the colorblind safe Okabe and Ito colormap to visualize the `group` attribute of the data points.

We can override the colormap using any Matplotlib color map or define our own categorical colormap easily using a dictionary of category-color pairs.

In [5]:
scatter.color(map=dict(
  C='red',
  B='blue',
  A='yellow',
  D='pink',
  E='green',
  F='brown',
  G='gray',
  H='#56B4E9'
))

# Short-Hand Test

If you only want to plot something once, you can short-hand API.

In [7]:
jscatter.plot(
    data=df,
    x='x',
    y='y',
    color_by='value',
    color_order='reverse',
    size_by='value',
    size_map=[10,11,12,13,14,15],
    opacity_by='value',
    connect_by='group',
    connection_color_by='value',
    connection_color_order='reverse',
    height=600
)

# Large-Scale Test

In the following we're going to visualize the Rössler Attractor using 1 million points. This example is taken from the [absolutely fantastic writeup by the wonderful Ricky Reusser](https://observablehq.com/@rreusser/selecting-the-right-opacity-for-2d-point-clouds). Ricky, you're the best!

In [8]:
def roesslerAttractor(num):
    from math import inf
    
    points = []

    xn = 2.644838333129883
    yn = 4.060488700866699
    zn = 2.8982460498809814
    a = 0.2
    b = 0.2
    c = 5.7
    dt = 0.006

    minX = inf
    maxX = -inf
    minY = inf
    maxY = -inf
    for i in range(num):
        dx = -yn - zn
        dy = xn + a * yn
        dz = b + zn * (xn - c)

        xh = xn + 0.5 * dt * dx
        yh = yn + 0.5 * dt * dy
        zh = zn + 0.5 * dt * dz

        dx = -yh - zh
        dy = xh + a * yh
        dz = b + zh * (xh - c)

        xn1 = xn + dt * dx
        yn1 = yn + dt * dy
        zn1 = zn + dt * dz

        points.append([xn1, yn1])

        minX = min(xn1, minX)
        maxX = max(xn1, maxX)
        minY = min(yn1, minY)
        maxY = max(yn1, maxY)

        xn = xn1;
        yn = yn1;
        zn = zn1;


    dX = maxX - minX
    dY = maxY - minY

    for i in range(num):
        points[i][0] -= minX
        points[i][0] /= dX / 2
        points[i][0] -= 1
        points[i][1] -= minY
        points[i][1] /= dY / 2
        points[i][1] -= 1

    return points

In [9]:
points = np.asarray(roesslerAttractor(1000000))
jscatter.plot(points[:,0], points[:,1], height=640)

Note, as you're zooming in, the point opacity is automatically adjusted based the number of points within the view. This feature is an extension from Ricky Reusser's dynamic point opacity introduced in [his fantastic writeup](https://observablehq.com/@rreusser/selecting-the-right-opacity-for-2d-point-clouds) mentioned above.